# Imports and Preprocessing

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from scikeras.wrappers import KerasRegressor

In [24]:
import os
import pandas as pd
folder_path = 'Data_Test_v1'
file_names = ['data_test.csv', 'data_train.csv', 'target_test.csv', 'target_train.csv']

data_frames = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    data_frames.append(df)

data_test = data_frames[0]
data_train = data_frames[1]
target_test = data_frames[2]    
target_train = data_frames[3]

   sales
0   1798
1   1291
2   1236
3   1829
4   2097


# ANN Model

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.optimizers import Adam

def create_model(n_hidden=1, n_neurons=30, input_shape=[3]): #learning_rate_input=3e-3, 
    model = Sequential()
    model.add(Dense(n_neurons, activation="relu", input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation="relu"))
    model.add(Dense(1))
    optimizer = Adam() #learning_rate=learning_rate_input
    model.compile(loss="mean_squared_error", optimizer=optimizer)
    return model

In [9]:
def model_builder(n_hidden=1, n_neurons=30, ): #learning_rate=3e-3
    return KerasRegressor(build_fn=create_model, verbose=0, n_hidden=n_hidden, n_neurons=n_neurons) #, learning_rate=learning_rate

In [10]:
from scipy.stats import reciprocal

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    #"learning_rate": reciprocal(3e-4, 3e-2),
}

In [11]:
model_ANN = model_builder()
rnd_search_cv_ANN = RandomizedSearchCV(model_ANN, param_distribs, n_iter=10, cv=3)

In [12]:
rnd_search_cv_ANN.fit(data_train, target_train)

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(build_fn=<function create_model at 0x00000205EA700F40>, n_hidden=1, n_neurons=30, verbose=0),
                   param_distributions={'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [13]:
target_pred_ANN = rnd_search_cv_ANN.predict(data_test)

# DT using LightGBM

In [14]:
import lightgbm as lgb

# Create the model
model_DT = lgb.LGBMRegressor()

# Train the model
model_DT.fit(data_train, target_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000108 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 51
[LightGBM] [Info] Number of data points in the train set: 208, number of used features: 3
[LightGBM] [Info] Start training from score 1528.524038
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

LGBMRegressor()

In [15]:
# Make predictions
target_pred_DT = model_DT.predict(data_test)

# Forecasting Error

In [16]:
# Calculate forecast error for ANN
train_pred_ANN = rnd_search_cv_ANN.predict(data_train)
train_pred_ANN_flatten = train_pred_ANN.flatten()
forecast_error_ANN = target_train - train_pred_ANN_flatten

In [17]:
# Calculate forecast error for DT
train_pred_DT = model_DT.predict(data_train)
train_pred_DT_flatten = train_pred_DT.flatten()
forecast_error_DT = target_train - train_pred_DT_flatten

# Predicting Order Quantities 

In [18]:
# Initialize an empty list to store the final order quantities
final_order_quantities_ANN = []
final_order_quantities_DT = []

# Define costs
c = 2  # cost per unit of product
s = 1  # salvage value per unit of leftover product
p = 3  # price per unit of product sold

# Calculate critical ratio
critical_ratio = (p - c) / ((p - s) + (p - c))

In [19]:
from scipy.stats import norm
# Loop over each week in data_test
for i in range(len(data_test)):

    # Number of scenarios
    num_scenarios = 1000

    # Generate demand scenarios for the week
    demand_scenarios_single = target_pred_ANN[i] + np.random.choice(forecast_error_ANN, size=num_scenarios)

    # Solve newsvendor problem for the week
    order_quantity_scenarios = norm.ppf(critical_ratio, loc=demand_scenarios_single)

    # Append the order quantity to the list
    final_order_quantities_ANN.append(np.mean(order_quantity_scenarios))

# Convert the list to a numpy array
final_order_quantities_ANN = np.array(final_order_quantities_ANN)

In [20]:
# Loop over each week in data_test
for i in range(len(data_test)):

    # Number of scenarios
    num_scenarios = 1000

    # Generate demand scenarios for the week
    demand_scenarios_single = target_pred_DT[i] + np.random.choice(forecast_error_DT, size=num_scenarios)

    # Solve newsvendor problem for the week
    order_quantity_scenarios = norm.ppf(critical_ratio, loc=demand_scenarios_single)

    # Append the order quantity to the list
    final_order_quantities_DT.append(np.mean(order_quantity_scenarios))

# Convert the list to a numpy array
final_order_quantities_DT = np.array(final_order_quantities_DT)

In [25]:
print(final_order_quantities_ANN)

[1514.48660221 1527.38327916 1522.60205743 1522.40135788 1530.26729113
 1512.02814448 1539.01841911 1545.48466314 1542.42466725 1532.32357008
 1519.86990304 1518.63585846 1528.03022972 1536.96873812 1550.50170274
 1524.41971823 1533.31878997 1534.1195066  1533.66823095 1520.72975546
 1520.49484855 1541.68270384 1527.05292981 1532.00766517 1540.41794274
 1532.07266827 1511.55586576 1546.76959721 1530.24960961 1530.09183207
 1526.19609079 1551.41277647 1525.89987536 1553.33511254 1521.49523812
 1508.27686615 1509.03215179 1517.91281671 1534.56632727 1517.91377397
 1534.56528222 1536.59278491 1529.23633073 1531.68482476 1513.68625558
 1537.9029     1524.51280772 1531.11542472 1527.4632694  1543.7887275
 1529.80810189 1513.95479515 1539.61699374]


# Calculate Costs

In [21]:
# Loop over each week in target_test
overall_profit_ANN = 0
overall_profit_DT = 0
for i in range(len(target_test)):

    # Calculate the profit for the week (without over or understock costs)
    profit_ANN = (p - c) * round(final_order_quantities_ANN[i])
    profit_DT = (p - c) * round(final_order_quantities_DT[i])

    # Calculate understock and overstock costs
    overstock_ANN = 0
    overstock_DT = 0
    understock_ANN = 0
    understock_DT = 0
    if final_order_quantities_ANN[i] < target_test.values[i]:
        understock_ANN = (p - c) * (target_test.values[i] - round(final_order_quantities_ANN[i]))
    if final_order_quantities_ANN[i] > target_test.values[i]:
        overstock_ANN = (c - s) * (round(final_order_quantities_ANN[i]) - target_test.values[i])
    if final_order_quantities_DT[i] < target_test.values[i]:
        understock_DT = (p - c) * (target_test.values[i] - round(final_order_quantities_DT[i]))
    if final_order_quantities_DT[i] > target_test.values[i]:
        overstock_DT = (c - s) * (round(final_order_quantities_DT[i]) - target_test.values[i])
    

    # Calculate the total profit for the week
    total_profit_ANN = profit_ANN - overstock_ANN - understock_ANN
    total_profit_DT = profit_DT - overstock_DT - understock_DT

    # Add the total profit to the overall profit
    overall_profit_ANN += total_profit_ANN
    overall_profit_DT += total_profit_DT
   

# Print the overall profit
print('Overall profit for ANN: ', overall_profit_ANN)
print('Overstock: ', overstock_ANN, ' Understock: ', understock_ANN)
print('Overall profit for DT: ', overall_profit_DT)
print('Overstock: ', overstock_DT, ' Understock: ', understock_DT)

Overall profit for ANN:  66901
Overstock:  0  Understock:  92
Overall profit for DT:  81205
Overstock:  33  Understock:  0
